# Uploading a synonym map

In [34]:
import requests
import json

In [48]:
service_name = "cog-search-jd5ypzfx2l6vi"
api_version = "2023-07-01-Preview"
admin_key = "omleyzSnaE1LguzkKUSTruoaCNTTIZKqwgrlCRYH1WAzSeCdPdhE"
language = "sv"

In [49]:
agresso_synonym_map={
    "name": "agresso-synonyms",
    "format": "solr",
    "synonyms": """
        agresso, marksaas, levop\n
        faktura, henke, miro\n"""
}

**uploading synonym map**

In [50]:
url = f"https://{service_name}.search.windows.net/synonymmaps?api-version={api_version}"

headers = {
    "Content-Type": "application/json",
    "api-key": admin_key,
}

# Replace this with your actual data
body = agresso_synonym_map

response = requests.post(url, headers=headers, data=json.dumps(body))

print(response.status_code)
print(response.text)

400
{"error":{"code":"ResourceNameAlreadyInUse","message":"Cannot create synonym map'agresso-synonyms' because a synonym map with that name already exists","details":[{"code":"DuplicateSynonymMap","message":"Cannot create synonym map'agresso-synonyms' because a synonym map with that name already exists"}]}}


**Assign synonym map to fields**

In [55]:
service_name = "cog-search-jd5ypzfx2l6vi"
api_version = "2023-07-01-Preview"
admin_key = "omleyzSnaE1LguzkKUSTruoaCNTTIZKqwgrlCRYH1WAzSeCdPdhE"
language = "sv"
index_name="001-test1-test2"

url = f"https://{service_name}.search.windows.net/indexes/{index_name}?api-version={api_version}"

headers = {
    "Content-Type": "application/json",
    "api-key": admin_key
}

body = {
    "fields": [
        {
            "name": "content",
            "type": "Edm.String",
            "searchable": True,
            "sortable": False,
            "facetable": False,
            "filterable": False,
            "analyzer": f"{language}.lucene" if language else None,
            "synonymMaps": ["agresso-synonyms"]
        }
    ]
}

response = requests.put(url, headers=headers, data=json.dumps(body))

# Check the response
if response.status_code == 200:
    print("Request was successful.")
else:
    print(f"Request failed. Status code: {response.status_code}")
    print(response.text)


Request failed. Status code: 400
{"error":{"code":"OperationNotAllowed","message":"The request is invalid. Details: index : Found 0 key fields in index '001-test1-test2'. Each index must have exactly one key field.","details":[{"code":"MissingKeyField","message":"Found 0 key fields in index '001-test1-test2'. Each index must have exactly one key field. Parameters: index"}]}}


In [63]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType

# Create a new SearchIndexClient
endpoint = f"https://{service_name}.search.windows.net"
credential = AzureKeyCredential(admin_key)
index_client = SearchIndexClient(endpoint, credential)

# Get the index
index = index_client.get_index(name=index_name)

# Assign the synonym map to a field
for field in index.fields:
    if field.name == "content":
        field.synonym_maps = ["agresso-synonyms"]

# Update the index
index_client.create_or_update_index(index)


id
content
title
filepath
url
metadata
contentVector


**list synonym maps**

In [68]:
url = f"https://{service_name}.search.windows.net/synonymmaps?api-version={api_version}"
headers = {
    'Content-Type': 'application/json',
    'api-key': admin_key
}

response = requests.get(url, headers=headers)

print(response.status_code)
# print(response.json())
print(f'Synonym map: {response.json()["value"][0]}')

200
Synonym map: {'@odata.etag': '"0x8DC0306A8297285"', 'name': 'agresso-synonyms', 'format': 'solr', 'synonyms': '\n        agresso, marksaas, levop\n\n        faktura, henke, miro\n', 'encryptionKey': None}


**Get specific synonym map**

In [23]:
synonymmap_name="agresso-synonyms"

url = f"https://{service_name}.search.windows.net/synonymmaps/{synonymmap_name}?api-version={api_version}"
headers = {
    'Content-Type': 'application/json',
    'api-key': admin_key
}

response = requests.get(url, headers=headers)

print(response.status_code)
print(response.json())

200
{'@odata.context': 'https://cog-search-jd5ypzfx2l6vi.search.windows.net/$metadata#synonymmaps/$entity', '@odata.etag': '"0x8DC0301F445C216"', 'name': 'agresso-synonyms', 'format': 'solr', 'synonyms': '\n        agresso, marksaas, levop\n\n        faktura, henke, miro\n', 'encryptionKey': None}


**delete a synonym map**

In [33]:
synonymmap_name="agresso-synonyms"

url = f"https://{service_name}.search.windows.net/synonymmaps/{synonymmap_name}?api-version={api_version}"
headers = {
    'Content-Type': 'application/json',
    'api-key': admin_key
}

response = requests.delete(url, headers=headers)

print(response.status_code)

404


In [ ]:
import requests
import json

url = "https://my-search-service.search.windows.net/indexes/my-index?api-version=2023-11-01&allowIndexDowntime=true"
headers = {
    "Content-Type": "application/json",
    "api-key": "{{admin-api-key}}",
}

body = {
    "name": "{{index-name}}",
    "fields": [
        {
            "name": "id",
            "type": "Edm.String",
            "key": True,
            "filterable": True
        },
        {
            "name": "title",
            "type": "Edm.String",
            "searchable": True,
            "filterable": True,
            "sortable": True,
            "retrievable": True
        },
        {
            "name": "titleVector",
            "type": "Collection(Edm.Single)",
            "searchable": True,
            "retrievable": True,
            "dimensions": 1536,
            "vectorSearchProfile": "my-default-vector-profile"
        },
        {
            "name": "content",
            "type": "Edm.String",
            "searchable": True,
            "retrievable": True
        },
        {
            "name": "contentVector",
            "type": "Collection(Edm.Single)",
            "searchable": True,
            "retrievable": True,
            "dimensions": 1536,
            "vectorSearchProfile": "my-default-vector-profile"
        }
    ],
    "vectorSearch": {
        "algorithms": [
            {
                "name": "my-hnsw-config-1",
                "kind": "hnsw",
                "hnswParameters": {
                    "m": 4,
                    "efConstruction": 400,
                    "efSearch": 500,
                    "metric": "cosine"
                }
            }
        ],
        "profiles": [
            {
                "name": "my-default-vector-profile",
                "algorithm": "my-hnsw-config-1"
            }
        ]
    }
}

response = requests.put(url, headers=headers, data=json.dumps(body))

# Check the response
if response.status_code == 200:
    print("Request was successful")
else:
    print(f"Request failed with status code {response.status_code}")